# [5-3] JETANK_2_ctrl

이 문서는 JETANK 제어 기능을 구축하는 데 사용되며, 페이지의 버튼을 사용하여 JETANK를 제어합니다. 또한 버튼에 대한 기능을 사용자 정의할 수 있습니다.

## 카메라 클래스를 가져오기
다음 코드를 실행하여 카메라 위젯을 엽니다. 화면이 열리면 화면을 마우스 오른쪽 버튼으로 클릭하여 "출력용 새 보기 만들기"를 선택한 다음, 화면을 적절한 위치로 끌어다 놓을 수 있습니다.

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from robotpal import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)
#카메라로 찍은 frame 을 띄워줄 image 객체 생성, 크기를 맞출 필요는 없다.
image = widgets.Image(format='jpeg', width=300, height=300) 

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998


Image(value=b'', format='jpeg', height='300', width='300')

[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)


[System] WebSocket 프로세서 시작


In [2]:
from robotpal.SCSCtrl import TTLServo
import time

# 그리퍼 위치 변수
xPos = 100
yPos = 0
servoPos_4 = 0

# 그리퍼 x 값 증가 함수
def xIn():
    global xPos
    xPos += 15
    TTLServo.xyInput(xPos, yPos)

# 그리퍼 x 값 감소 함수
def xDe():
    global xPos
    xPos -= 15
    if xPos < 85:
        xPos = 85
    TTLServo.xyInput(xPos, yPos)

# 그리퍼 y 값 증가 함수
def yIn():
    global yPos
    yPos += 15
    TTLServo.xyInput(xPos, yPos)

# 그리퍼 y 값 감소 함수
def yDe():
    global yPos
    yPos -= 15
    TTLServo.xyInput(xPos, yPos)

# 그리퍼 집기 함수
def grab():
    global servoPos_4
    servoPos_4 -= 15
    if servoPos_4 < -90:
        servoPos_4 = -90
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)
    
# 그리퍼 풀기 함수
def loose():
    global servoPos_4
    servoPos_4+= 15
    if servoPos_4 > -10:
        servoPos_4 = -10
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)

# 그리퍼 제어용 버튼 배치 및 생성 코드
arm_layout = widgets.Layout(width='100px', height='80px', align_self='center')
xIn_button = widgets.Button(description='X+', button_style='success', layout=arm_layout)
xDe_button = widgets.Button(description='X-', button_style='success', layout=arm_layout)
yIn_button = widgets.Button(description='Y+', button_style='success', layout=arm_layout)
yDe_button = widgets.Button(description='Y-', button_style='success', layout=arm_layout)
grab_button = widgets.Button(description='Grab', layout=arm_layout)
loose_button = widgets.Button(description='Loose', layout=arm_layout)

line1_box = widgets.HBox([xIn_button, yIn_button])
line2_box = widgets.HBox([xDe_button, yDe_button])
line3_box = widgets.HBox([grab_button, loose_button])
arm_box = widgets.VBox([line1_box, line2_box, line3_box])

display(arm_box)


In [3]:
xIn_button.on_click(lambda x: xIn())
xDe_button.on_click(lambda x: xDe())

yIn_button.on_click(lambda x: yIn())
yDe_button.on_click(lambda x: yDe())

grab_button.on_click(lambda x: grab())
loose_button.on_click(lambda x: loose())

def grab_object():

    # GRAB
    # Position arm down
    TTLServo.xyInput(200, 90)
    time.sleep(1)
    # Close grip
    TTLServo.servoAngleCtrl(4, 40, -1, 150)
    time.sleep(5)

    # PLACE IN BASKET
    # Position arm at back
#     TTLServo.servoAngleCtrl(2, 100, -1, 200)
#     TTLServo.servoAngleCtrl(3, 100, 1, 200)
    TTLServo.xyInput(125, 130)
    time.sleep(5)
    # Open grib
#         TTLServo.servoAngleCtrl(4, 100, 1, 150)
    time.sleep(1)
    # Position arm at initial position
    TTLServo.servoAngleCtrl(2, 0, 1, 200)
    TTLServo.servoAngleCtrl(3, 0, 1, 200)

In [4]:
# Grap function
def grab():
    
    # GRAB
    # Position arm down
    TTLServo.xyInput(200, -90)
    time.sleep(1)
    # Close grip
    # TTLServo.servoAngleCtrl(4, 100, -1, 150)
    time.sleep(5)
    
    # PLACE IN BASKET
    # Position arm at back
    TTLServo.servoAngleCtrl(2, 100, -1, 200)
    TTLServo.servoAngleCtrl(3, 100, 1, 200)
    time.sleep(5)
    # Open grib
    # TTLServo.servoAngleCtrl(4, 100, 1, 150)
    time.sleep(3)
    # Position arm at initial position
    TTLServo.servoAngleCtrl(2, 0, 1, 200)
    TTLServo.servoAngleCtrl(3, 0, 1, 200)

In [5]:
grab()

In [8]:
TTLServo.servoAngleCtrl(1, 0, 1, 200)
TTLServo.servoAngleCtrl(2, -90, 1, 200)
TTLServo.servoAngleCtrl(3, 0, 1, 200)
TTLServo.servoAngleCtrl(4, 0, 1, 200)
TTLServo.servoAngleCtrl(5, 45, 1, 200)

724

다시 한 번 카메라 연결을 적절하게 종료하여 나중에 노트북에서 카메라를 사용할 수 있도록 합시다.

In [2]:
camera.stop()